In [ ]:
!pip install acumos
!conda install -y libprotobuf

In [ ]:
import os
import os.path
import time

from acumos.session import AcumosSession
from acumos.modeling import Model, List, create_dataframe
 
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X = iris.data
y = iris.target
 
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)

# here, an appropriate NamedTuple type is inferred from a pandas DataFrame
X_df = pd.DataFrame(X, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
IrisDataFrame = create_dataframe('IrisDataFrame', X_df)
 
# ==================================================================================
# # or equivalently:
#
# IrisDataFrame = create_namedtuple('IrisDataFrame', [('sepal_length', List[float]),
#                                                     ('sepal_width', List[float]),
#                                                     ('petal_length', List[float]),
#                                                     ('petal_width', List[float])])
# ==================================================================================
 
def classify_iris(df: IrisDataFrame) -> List[int]:
    '''Returns an array of iris classifications'''
    X = np.column_stack(df)
    return clf.predict(X)
 
model = Model(classify=classify_iris)
 
session = AcumosSession()

if os.path.exists("iris_sklearn"):
    os.remove('iris_sklearn/metadata.json')
    os.remove('iris_sklearn/model.proto')
    os.remove('iris_sklearn/model.zip')
    os.rmdir('iris_sklearn')

session.dump(model,'iris_sklearn','~/.')

while not os.path.exists("iris_sklearn"):
    time.sleep(1)

print("The trained model has been dumped into the iris_sklearn folder")

In [ ]:
if not os.path.exists("acumos-push.py"):
    !wget https://wiki.acumos.org/download/attachments/17694974/acumos-push.py
os.environ['ACUMOS_USERNAME'] = "test"
os.environ['ACUMOS_PASSWORD'] = "P@ssw0rd"
os.environ['ACUMOS_TOKEN'] = "171cfda8dfa24edf8dfbc3dd681d8562"
os.environ['ACUMOS_DUMPDIR'] = "iris_sklearn"
!python acumos-push.py